# SymPrompt Demo

SymPrompt is a two-tier neuro-symbolic framework for translating natural language prompts into formal symbolic representations (SymIL) and verifying them with solvers.

This notebook demonstrates:
1. Basic syllogistic reasoning
2. Multi-domain examples (math, planning, legal)
3. The routing system and tier selection
4. SymIL representation inspection

In [ ]:
# Setup - import required modules
from symprompt.integration.router_adapter import route_and_solve
from symprompt.llm.sync_client import build_default_sync_client
from symprompt.router.smart_router import SmartRouter
from symprompt.translation.pipeline import TranslationPipeline

# Initialize components
llm_client = build_default_sync_client()
router = SmartRouter()
pipeline = TranslationPipeline.from_llm_client(llm_client)

print("SymPrompt initialized successfully!")

## 1. Basic Syllogistic Reasoning

Classic categorical syllogisms are the foundation of formal logic.

In [ ]:
# Classic Socrates syllogism
prompt = "All men are mortal. Socrates is a man. Is Socrates mortal?"

result = route_and_solve(prompt, llm_client)

print(f"Prompt: {prompt}")
print(f"\nRouting Decision:")
print(f"  Tier: {result.routing.tier}")
print(f"  Profile: {result.routing.profile_name}")
print(f"  SymIL Level: {result.routing.symil_level}")
print(f"\nSolver Result: {result.solver_result.get('status')}")

In [ ]:
# View the SymIL representation
if result.symil:
    print("SymIL Representation:")
    print(f"\nFacts:")
    for fact in result.symil.facts:
        print(f"  {fact}")
    print(f"\nRules:")
    for rule in result.symil.rules:
        print(f"  {rule}")
    print(f"\nQuery: {result.symil.query}")

In [ ]:
# Invalid syllogism - affirming the consequent fallacy
prompt = "All cats are mammals. Fido is a mammal. Is Fido a cat?"

result = route_and_solve(prompt, llm_client)

print(f"Prompt: {prompt}")
print(f"Solver Result: {result.solver_result.get('status')}")
print(f"(Expected: NOT_VALID - this is the affirming the consequent fallacy)")

## 2. Mathematical Reasoning

SymPrompt handles mathematical relationships using Z3 solver.

In [ ]:
# Transitive inequality
prompt = "If x > y and y > z, then x > z. x is 5, y is 3, z is 1. Is x greater than z?"

result = route_and_solve(prompt, llm_client)

print(f"Prompt: {prompt}")
print(f"Profile: {result.routing.profile_name}")
print(f"Solver Result: {result.solver_result.get('status')}")

In [ ]:
# Invalid mathematical inference
prompt = "a is greater than b. c is greater than d. Is a greater than d?"

result = route_and_solve(prompt, llm_client)

print(f"Prompt: {prompt}")
print(f"Solver Result: {result.solver_result.get('status')}")
print(f"(Expected: NOT_VALID - no transitive chain between a and d)")

## 3. Planning Domain

Planning problems involve preconditions, actions, and effects.

In [ ]:
# Simple precondition check
prompt = "To open a door, you need a key. Alice has a key. Can Alice open the door?"

result = route_and_solve(prompt, llm_client)

print(f"Prompt: {prompt}")
print(f"Profile: {result.routing.profile_name}")
print(f"Tier: {result.routing.tier}")
print(f"Solver Result: {result.solver_result.get('status')}")

In [ ]:
# Resource constraint
prompt = "Building a house requires 100 bricks. We have 50 bricks. Can we build a house?"

result = route_and_solve(prompt, llm_client)

print(f"Prompt: {prompt}")
print(f"Solver Result: {result.solver_result.get('status')}")
print(f"(Expected: NOT_VALID - insufficient resources)")

## 4. Legal/Deontic Reasoning

Legal reasoning involves obligations, permissions, and prohibitions.

In [ ]:
# Permission
prompt = "Adults are permitted to vote. Sarah is an adult. Is Sarah permitted to vote?"

result = route_and_solve(prompt, llm_client)

print(f"Prompt: {prompt}")
print(f"Profile: {result.routing.profile_name}")
print(f"Solver Result: {result.solver_result.get('status')}")

In [ ]:
# Exception handling
prompt = "All vehicles must pay tolls, except emergency vehicles. An ambulance is an emergency vehicle. Must an ambulance pay tolls?"

result = route_and_solve(prompt, llm_client)

print(f"Prompt: {prompt}")
print(f"Tier: {result.routing.tier}")
print(f"Solver Result: {result.solver_result.get('status')}")
print(f"(Expected: NOT_VALID - exception applies)")

## 5. Batch Evaluation

Run multiple prompts and check accuracy.

In [ ]:
test_cases = [
    ("All birds can fly. Tweety is a bird. Can Tweety fly?", "VALID"),
    ("All squares are rectangles. This shape is a rectangle. Is it a square?", "NOT_VALID"),
    ("If it rains, the ground is wet. It rained. Is the ground wet?", "VALID"),
    ("Some dogs are friendly. Max is a dog. Is Max friendly?", "NOT_VALID"),
]

correct = 0
for prompt, expected in test_cases:
    result = route_and_solve(prompt, llm_client)
    actual = result.solver_result.get('status')
    match = "OK" if actual == expected else "MISMATCH"
    if actual == expected:
        correct += 1
    print(f"[{match}] {prompt[:50]}...")
    print(f"    Expected: {expected}, Got: {actual}")

print(f"\nAccuracy: {correct}/{len(test_cases)} ({100*correct//len(test_cases)}%)")

## 6. Router Feature Extraction

Inspect how the router classifies prompts.

In [ ]:
from symprompt.router.smart_router import SmartRouter

router = SmartRouter()

prompts = [
    "All mammals are animals. Dogs are mammals. Are dogs animals?",
    "x + y = 10. x = 3. What is y?",
    "To bake a cake, you need flour. We have flour. Can we bake?",
    "Employees must report by 9am. John arrived at 8:30am. Did John comply?",
]

for prompt in prompts:
    decision = router.route(prompt)
    print(f"Prompt: {prompt[:50]}...")
    print(f"  -> Tier: {decision.tier}, Profile: {decision.profile_name}, Level: {decision.symil_level}")
    print()

## Summary

SymPrompt provides:
- **Two-tier routing**: Fast path (Tier 1) for simple logic, full pipeline (Tier 2) for complex reasoning
- **Multi-domain support**: Syllogism, math, planning, legal, and more
- **SymIL representation**: Structured intermediate language for formal reasoning
- **Multiple solvers**: Z3, Clingo (ASP), Scallop, VSA

For more information, see the architecture documentation in `docs/`.